In [1]:
import pandapower as pp
import pandas as pd
import numpy as np
from pandapower.timeseries.output_writer import OutputWriter


# Install pandapower in a new environment by pip install pandapower==2.13.1
net = pp.from_pickle('./aggregated_grid_2018_with_generators_loads_costs_controllers.p')

out_path = "./AC_OPF"
ow = OutputWriter(net, output_path=out_path, output_file_type=".csv", csv_separator=",")
# ow.remove_log_variable("res_gen", "vm_pu")

In [26]:
# TIMESTEP-WISE SIMULATION TO GET SIMULATED VALUES


# Single time values
def get_simulated_values(t):
    for controller in net.controller.object:
        controller.time_step(t, net)

    pp.runpp(net)

In [27]:
# GET FROM SIMULATION

def get_from_simulation(res_csv, component_numbers):
    # Join simulation timeseries dataframes in a dataframe
    bus_timesteps = pd.concat([res_csv[f"{g}"] for g in component_numbers], axis=1)

    return bus_timesteps

In [28]:
# GET FROM CONTROLLER

def get_from_controller(start_controller_number, component_numbers):
    # Join controller timeseries dataframes at the same bus in a dataframe
    bus_timesteps = pd.concat([net.controller.object[start_controller_number + g].data_source.to_dict()['df'].iloc[:,0] for g in component_numbers], axis=1)

    return bus_timesteps

In [29]:
# GET AGGREGATED DATA

total_bus_numbers = net.bus.index

def get_aggregated_x(res_x, bus_gens, get_from):
    # res_x = csv for get_from_csv or start controller number for get_from_controller
    gen_busses_x = [] # List of timestep-wise gen x dataframes for each bus

    for bus in bus_gens:
        # Join timeseries dataframes (q_mvar, vm_pu and va_degree) at the same bus in a dataframe
        bus_timesteps = get_from(res_x, bus)

        # Add timestep-wise values together
        bus_timesteps_total = bus_timesteps.sum(axis=1)
        
        gen_busses_x.append(bus_timesteps_total)

    gen_x = pd.concat(gen_busses_x, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)

    gen_x.columns = bus_gens.index # Column names to bus numbers

    gen_x = gen_x.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
    
    return gen_x



# gen_busses_q_mvar = [] 
# gen_busses_vm_pu = []
# gen_busses_va_degree = []



# for bus in bus_gens:
#     # Join timeseries dataframes (q_mvar, vm_pu and va_degree) at the same bus in a dataframe
#     bus_timesteps = pd.concat([res_q_mvar[f"{g}"] for g in bus], axis=1)
#     bus_timesteps_vm = pd.concat([res_vm_pu[f"{vm}"] for vm in bus], axis=1)
#     bus_timesteps_va = pd.concat([res_va_degree[f"{va}"] for va in bus], axis=1)

#     # Add timestep-wise values together
#     bus_timesteps_total = bus_timesteps.sum(axis=1)
#     bus_timesteps_total_vm = bus_timesteps_vm.sum(axis=1)
#     bus_timesteps_total_va = bus_timesteps_va.sum(axis=1)

#     gen_busses_x.append(bus_timesteps_total)
#     gen_busses_vm_pu.append(bus_timesteps_total_vm)    
#     gen_busses_va_degree.append(bus_timesteps_total_va)

# gen_q_mvar = pd.concat(gen_busses_x, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)
# gen_vm_pu = pd.concat(gen_busses_vm_pu, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)
# gen_va_degree = pd.concat(gen_busses_va_degree, axis=1) # Put busses together in one df (rows = timesteps, columns = buses)

# gen_q_mvar.columns = bus_gens.index # Column names to bus numbers
# gen_vm_pu.columns = bus_gens.index # Column names to bus numbers
# gen_va_degree.columns = bus_gens.index # Column names to bus numbers

# gen_q_mvar = gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
# gen_vm_pu = gen_vm_pu.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0
# gen_va_degree = gen_va_degree.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), axis=1, fill_value=0) # Fill missing bus numbers with q_mvar = 0

# gen_vm_pu.head(100)    


In [30]:
# CREATE LOAD Q_MVAR

pf = 0.95 # Using a power factor 0.95
p_to_q = np.sin(np.arccos(pf))/pf # P to Q constant

loads_p_mw = get_from_controller(0, net.load.index) # All controller p_mw data

loads_q_mvar = loads_p_mw.mul(p_to_q)

# loads_q_mvar.head(50)


In [31]:
# CREATE LOAD Q_MVAR CONTROLLER

for l in range(len(net.load.index)):
    controller_load_q_mvar = loads_p_mw.iloc[:, l].multiply(p_to_q)
    dataset = controller_load_q_mvar.to_frame()
    prof_name = dataset.columns[0]
    dataset = pp.timeseries.data_sources.frame_data.DFData(dataset)
    
    pp.control.controller.const_control.ConstControl(net, "load", "q_mvar", element_index=l, 
                                                 data_source=dataset, 
                                                 recycle={'trafo': False, 'gen': False, 'bus_pq': True}, 
                                                 profile_name=prof_name)

# net.controller

In [17]:
# SET REASONABLE Q_MVAR BOUNDARIES FOR GEN

for i in net.gen.index:
    net.gen.loc[i, "min_q_mvar"] = -1000.0
    net.gen.loc[i, "max_q_mvar"] = 1000.0

In [32]:
# SET REASONABLE Q_MVAR BOUNDARIES FOR EXT_GRID

for i in net.ext_grid.index:
    net.ext_grid.loc[i, "min_q_mvar"] = -5000.0
    net.ext_grid.loc[i, "max_q_mvar"] = 5000.0

net.ext_grid.loc[0, "type"] = "External Grid" # Fix ext_grid 0
net.ext_grid.loc[0, "p_mw"] = 0.0

# net.ext_grid = net.ext_grid.drop([i for i in net.ext_grid.index]) # Empty net.ext_grid

# net.ext_grid

In [24]:
# LOG SIMULATION VARIABLES

ow.log_variable('res_bus', 'vm_pu') # Add logging for Bus
ow.log_variable('res_bus', 'va_degree') 
ow.log_variable('res_bus', 'p_mw') 
ow.log_variable('res_bus', 'q_mvar')

ow.log_variable('res_line', 'loading_percent') # Line

ow.log_variable('res_gen', 'p_mw') # Gen
ow.log_variable('res_gen', "q_mvar")
ow.log_variable('res_gen', "va_degree")
ow.log_variable('res_gen', "vm_pu")

ow.log_variable('res_sgen', 'p_mw') # Sgen
ow.log_variable('res_sgen', 'q_mvar')

ow.log_variable('res_load', 'p_mw') # Load
ow.log_variable('res_load', 'q_mvar')

ow.log_variable('res_ext_grid', 'p_mw') # Ext grid
ow.log_variable('res_ext_grid', 'q_mvar')

ow.log_variable('res_trafo', 'loading_percent') # Transformer
ow.log_variable('res_trafo', 'p_hv_mw')
ow.log_variable('res_trafo', 'p_hv_mvar')
ow.log_variable('res_trafo', 'p_lv_mw')
ow.log_variable('res_trafo', 'q_lv_mvar')
ow.log_variable('res_trafo', 'pl_mw')
ow.log_variable('res_trafo', 'ql_mvar')
ow.log_variable('res_trafo', 'i_hv_ka')
ow.log_variable('res_trafo', 'i_lv_ka')
ow.log_variable('res_trafo', 'vm_hv_pu')
ow.log_variable('res_trafo', 'va_hv_degree')
ow.log_variable('res_trafo', 'vm_lv_pu')
ow.log_variable('res_trafo', 'va_lv_degree')

# def get_simulated_timeseries(t_init, t_total):
#     pp.timeseries.run_time_series.run_timeseries(net, time_steps=range(t_init,t_init + t_total))

# get_simulated_timeseries(0, 24)

In [25]:
# GET SIMULATION RESULTS

pp.timeseries.run_timeseries(net, run=pp.runopp, continue_on_divergence=True) # Ignore divergence 
# pp.timeseries.run_timeseries(net, run=pp.runopp, continue_on_divergence=True, time_steps=range(0,24))

c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\build_gen.py:128: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  eg_constrained = net.ext_grid[eg_is][~net.ext_grid.controllable]
c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\opf\validate_opf_input.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  net[element_type].controllable.fillna(True, inplace=True)
c:\Users\bjorn\miniconda3\lib\site-packages\pandapower\opf\validate_opf_input.py:27: FutureWarning: A value is trying to be set

In [55]:
# SWAPPING SLACK BUS (35)

def swap_slack(df, slack_number): # Switch slack bus position

    copy = df.iloc[:, 0].copy()
    df.iloc[:, 0] = df.iloc[:, slack_number]
    df.iloc[:, slack_number] = copy
    
slack_bus = 35

In [56]:
# GET GENERATION P_MW

gen_active = net.gen.loc[net.gen["in_service"] == True] # Drop unused generators
bus_gens = gen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group generators by bus

res_gen_p_mw = pd.read_csv(out_path + "/res_gen/p_mw.csv") # Read from simulation result

gen_p_mw = get_aggregated_x(res_gen_p_mw, bus_gens, get_from_simulation) # Get bus-aggregated generation

swap_slack(gen_p_mw, slack_bus)

# gen_p_mw.head(50)

C:\Users\bjorn\AppData\Local\Temp\ipykernel_37492\3636657091.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_gens = gen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group generators by bus


In [57]:
# GET BUS VM_PU

res_vm_pu = pd.read_csv(out_path + "/res_bus/vm_pu.csv")

vm_pu = res_vm_pu.drop(columns=["Unnamed: 0"])

swap_slack(vm_pu, slack_bus)

vm_pu.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,1.050000,1.048237,1.049461,1.050000,1.047128,1.049809,1.049990,1.049905,1.049701,1.050000,1.049045,1.05,1.049020,1.048926,1.049399,1.049176,1.045282,1.043166,1.050000,1.046313,1.049336,1.05,1.049093,1.05,1.049249,1.05,1.045644,1.049252,1.05,1.048651,1.049443,1.048784,1.046186,1.048590,1.047169,1.05,1.05,1.050000,1.05,1.049998,1.048124,1.045644,1.045644,1.045644,1.049443
1,1.050000,1.048059,1.049512,1.050000,1.047095,1.049675,1.049990,1.049896,1.049704,1.050000,1.048344,1.05,1.048779,1.048492,1.048913,1.049082,1.045264,1.043519,1.050000,1.046078,1.049388,1.05,1.048706,1.05,1.048747,1.05,1.045351,1.049209,1.05,1.048502,1.049166,1.048488,1.046020,1.048231,1.047373,1.05,1.05,1.049999,1.05,1.049990,1.048080,1.045351,1.045351,1.045351,1.049166
2,1.050000,1.047992,1.049534,1.050000,1.047084,1.049542,1.049990,1.049892,1.049702,1.050000,1.047881,1.05,1.048714,1.048275,1.048602,1.049094,1.045268,1.043929,1.050000,1.045767,1.049483,1.05,1.048202,1.05,1.048354,1.05,1.045038,1.049209,1.05,1.048507,1.048979,1.048380,1.046095,1.048103,1.047699,1.05,1.05,1.050000,1.05,1.049994,1.048052,1.045038,1.045038,1.045038,1.048979
3,1.050000,1.047991,1.049558,1.050000,1.047091,1.049447,1.049990,1.049892,1.049727,1.050000,1.048205,1.05,1.049040,1.048573,1.048809,1.049361,1.045303,1.044276,1.050000,1.045471,1.049555,1.05,1.048051,1.05,1.048599,1.05,1.044636,1.049212,1.05,1.048855,1.048924,1.048695,1.046502,1.048408,1.047918,1.05,1.05,1.049999,1.05,1.049996,1.048165,1.044636,1.044636,1.044636,1.048924
4,1.050000,1.048068,1.049532,1.050000,1.047097,1.049398,1.049990,1.049885,1.049725,1.050000,1.048179,1.05,1.049013,1.048565,1.048798,1.049310,1.045313,1.044340,1.050000,1.045500,1.049597,1.05,1.048042,1.05,1.048585,1.05,1.044693,1.049234,1.05,1.048821,1.048908,1.048673,1.046526,1.048410,1.048035,1.05,1.05,1.049999,1.05,1.049994,1.048116,1.044693,1.044693,1.044693,1.048908
5,1.050000,1.048129,1.049481,1.050000,1.047063,1.049357,1.049990,1.049878,1.049701,1.050000,1.048016,1.05,1.048847,1.048415,1.048669,1.049162,1.045312,1.044306,1.050000,1.045591,1.049624,1.05,1.047933,1.05,1.048431,1.05,1.044935,1.049256,1.05,1.048653,1.048829,1.048508,1.046397,1.048278,1.048116,1.05,1.05,1.050000,1.05,1.049999,1.048008,1.044935,1.044935,1.044935,1.048829
6,1.050000,1.048176,1.049431,1.050000,1.046986,1.049354,1.049990,1.049874,1.049668,1.050000,1.047984,1.05,1.048748,1.048408,1.048718,1.049048,1.045375,1.044220,1.050000,1.045949,1.049618,1.05,1.048518,1.05,1.048502,1.05,1.045310,1.049262,1.05,1.048532,1.048989,1.048425,1.046324,1.048252,1.048085,1.05,1.05,1.049999,1.05,1.049994,1.048000,1.045310,1.045310,1.045310,1.048989
7,1.050000,1.048277,1.049386,1.050000,1.046921,1.049356,1.049990,1.049866,1.049636,1.050000,1.048383,1.05,1.048795,1.048581,1.048945,1.049052,1.045378,1.044084,1.050000,1.046095,1.049611,1.05,1.048840,1.05,1.048789,1.05,1.045518,1.049272,1.05,1.048542,1.049167,1.048496,1.046370,1.048377,1.048034,1.05,1.05,1.050000,1.05,1.049994,1.047986,1.045518,1.045518,1.045518,1.049167
8,1.050000,1.048291,1.049395,1.050000,1.046924,1.049332,1.049990,1.049861,1.049638,1.050000,1.048445,1.05,1.048732,1.048555,1.048959,1.049000,1.045363,1.043926,1.050000,1.046102,1.049566,1.05,1.048845,1.05,1.048811,1.05,1.045528,1.049267,1.05,1.048460,1.049172,1.048444,1.046255,1.048329,1.047914,1.05,1.05,1.049999,1.05,1.049995,1.048043,1.045528,1.045528,1.045528,1.049172
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [58]:
# GET LOAD P_MW

bus_loads = net.load.groupby("bus").apply(lambda x: x.index.tolist()) # Group loads by bus

load_p_mw = get_aggregated_x(0, bus_loads, get_from_controller)

# # Print load P_MW for forecasting:
# load_p_mw.to_csv(out_path + "/total_load.csv")

swap_slack(load_p_mw, slack_bus)

# load_p_mw.head(50)

C:\Users\bjorn\AppData\Local\Temp\ipykernel_37492\2619387209.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_loads = net.load.groupby("bus").apply(lambda x: x.index.tolist()) # Group loads by bus


In [59]:
# GET LOAD Q_MVAR

load_q_mvar = load_p_mw.mul(p_to_q)

# load_q_mvar.head(50)

In [60]:
# GET SGEN P_MW

sgen_active = net.sgen.loc[net.sgen["in_service"] == True] # Drop unused sgens
bus_sgens = sgen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group sgens by bus

res_sgen_p_mw = pd.read_csv(out_path + "/res_sgen/p_mw.csv") # Read from simulation result

sgen_p_mw = get_aggregated_x(res_sgen_p_mw, bus_sgens, get_from_simulation) # Get bus aggregated sgen

swap_slack(sgen_p_mw, slack_bus)

sgen_p_mw.head(50)

C:\Users\bjorn\AppData\Local\Temp\ipykernel_37492\723073240.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_sgens = sgen_active.groupby("bus").apply(lambda x: x.index.tolist()) # Group sgens by bus


bus,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0,0,0,186.789763,0,388.847995,348.181335,0,60.321648,18.121242,28.567543,260.179857,224.327865,127.905460,0,118.026561,0,66.380315,0,0,0,564.805929,177.509427,0,0,576.122375,10.564361,0,0,299.700172,66.630363,0,303.159788,0,36.665189,0,0,0,0,0,0,0,0,0,0
1,0,0,0,186.236025,0,386.373988,372.127424,0,52.085116,19.296122,29.862432,253.852046,222.915551,128.072775,0,118.245888,0,65.200416,0,0,0,587.079077,178.029199,0,0,579.896503,10.661393,0,0,294.872571,65.939232,0,302.169249,0,37.761231,0,0,0,0,0,0,0,0,0,0
2,0,0,0,179.981148,0,364.767344,373.358130,0,48.664880,18.893172,28.741387,251.693176,217.793319,127.324197,0,116.708836,0,62.862264,0,0,0,594.880740,176.276792,0,0,558.384886,10.772574,0,0,278.546820,66.182385,0,297.849227,0,36.525780,0,0,0,0,0,0,0,0,0,0
3,0,0,0,172.398325,0,340.807767,367.669088,0,55.242208,18.292662,26.511919,252.838922,214.513874,126.750577,0,113.685286,0,59.647632,0,0,0,594.617733,165.522906,0,0,528.717820,10.696683,0,0,264.458963,62.935551,0,291.389210,0,34.813263,0,0,0,0,0,0,0,0,0,0
4,0,0,0,168.347222,0,314.884383,356.371385,0,58.592960,17.464967,23.687744,249.562791,213.284817,126.322509,0,109.086288,0,53.678250,0,0,0,595.436474,151.224444,0,0,492.124761,10.144739,0,0,257.263128,58.606867,0,284.090493,0,32.456817,0,0,0,0,0,0,0,0,0,0
5,0,0,0,168.969859,0,297.971869,348.211726,0,59.859040,16.131720,24.147662,225.607587,213.236529,126.200673,0,107.194069,0,50.060738,0,0,0,595.809102,152.481471,0,0,460.651295,9.674330,0,0,257.598685,56.356515,0,284.157237,0,29.137268,0,0,0,0,0,0,0,0,0,0
6,0,0,0,169.900318,0,297.024384,349.571903,0,59.964500,15.297814,23.554438,188.754184,209.420472,125.131406,0,106.232998,0,47.800381,0,0,0,595.485034,138.152540,0,0,455.606080,9.084574,0,0,254.618583,47.656272,0,273.561694,0,28.105558,0,0,0,0,0,0,0,0,0,0
7,0,0,0,165.945548,0,286.874598,350.036002,0,60.056217,14.934291,21.803057,169.433605,207.165208,124.537565,0,103.194280,0,45.497594,0,0,0,595.843939,120.606118,0,0,440.283574,8.117401,0,0,241.282111,39.753723,0,262.164858,0,27.909286,0,0,0,0,0,0,0,0,0,0
8,0,0,0,172.850739,0,276.033550,361.002630,0,60.226098,28.106026,30.222009,185.344797,207.426541,124.781897,0,101.606278,0,70.147553,0,0,0,595.483670,166.969041,0,0,433.155764,35.358700,0,0,243.976079,42.017765,0,278.000989,0,47.382401,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0,0.000000,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0,0.000000,0,0.000000,0,0,0,0,0,0,0,0,0,0


In [61]:
# GET SGEN Q_MVAR

res_sgen_q_mvar = pd.read_csv(out_path + "/res_sgen/q_mvar.csv") # Read from simulation result

sgen_q_mvar = get_aggregated_x(res_sgen_q_mvar, bus_sgens, get_from_simulation) # Get bus aggregated sgen

swap_slack(sgen_q_mvar, slack_bus)

sgen_q_mvar.head(50)

bus,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0,0,0,1.444012e-17,0,7.875669e-16,8.901513e-18,0,3.339408e-18,1.059105e-17,1.783429e-17,3.782889e-18,7.356842e-18,8.123180e-19,0,2.832220e-18,0,8.318117e-15,0,0,0,-4.607035e-19,2.138799e-17,0,0,2.342583e-18,6.336345e-17,0,0,5.317886e-17,3.433205e-18,0,5.655346e-16,0,2.622157e-15,0,0,0,0,0,0,0,0,0,0
1,0,0,0,-5.140400e-15,0,1.659441e-14,-3.161886e-15,0,-2.018810e-15,9.769244e-16,-6.442996e-15,-2.001039e-15,-2.779188e-15,4.250555e-16,0,-5.468451e-16,0,-4.092022e-12,0,0,0,-5.241592e-17,-8.110441e-15,0,0,-1.316299e-15,-3.026421e-14,0,0,-2.124085e-14,-1.922173e-15,0,-1.110410e-12,0,8.514441e-13,0,0,0,0,0,0,0,0,0,0
2,0,0,0,-3.623051e-14,0,-2.444087e-13,-7.912571e-15,0,-1.004721e-14,-3.130112e-15,-2.441264e-14,-7.771312e-15,-1.533391e-14,1.568334e-14,0,6.153017e-15,0,-2.402624e-11,0,0,0,5.619691e-16,-3.673340e-14,0,0,8.595468e-15,-1.310107e-13,0,0,-1.135146e-13,-7.660520e-15,0,-9.901785e-12,0,5.257613e-12,0,0,0,0,0,0,0,0,0,0
3,0,0,0,5.127042e-16,0,1.272023e-14,7.618689e-16,0,2.393874e-16,-4.822047e-16,1.016191e-15,2.894384e-16,5.301982e-16,-5.943555e-17,0,3.499056e-17,0,5.004706e-13,0,0,0,2.439297e-18,1.156993e-15,0,0,-3.291742e-16,4.246920e-15,0,0,3.188568e-15,3.597185e-16,0,1.197486e-13,0,-9.785887e-14,0,0,0,0,0,0,0,0,0,0
4,0,0,0,-3.506284e-15,0,-4.346513e-14,-3.116910e-15,0,-1.441324e-15,-3.634431e-15,-3.450001e-15,-1.514160e-15,-2.426543e-15,-5.414134e-17,0,-5.108295e-16,0,-1.526928e-12,0,0,0,-1.972421e-16,-5.287201e-15,0,0,-2.604262e-15,-1.606360e-14,0,0,-1.133293e-14,-1.655013e-15,0,-1.652240e-13,0,3.055921e-13,0,0,0,0,0,0,0,0,0,0
5,0,0,0,2.543128e-15,0,3.783768e-14,2.248520e-15,0,1.059914e-15,2.613113e-15,2.605363e-15,1.095567e-15,1.834581e-15,1.203124e-16,0,3.701716e-16,0,1.160186e-12,0,0,0,1.176070e-16,3.939167e-15,0,0,1.861045e-15,1.219919e-14,0,0,8.597680e-15,1.235739e-15,0,1.411548e-13,0,-2.185312e-13,0,0,0,0,0,0,0,0,0,0
6,0,0,0,1.300238e-16,0,2.006618e-15,1.146737e-16,0,5.450110e-17,1.341812e-16,1.347810e-16,5.589210e-17,9.503436e-17,8.550188e-18,0,1.904884e-17,0,6.272024e-14,0,0,0,5.545421e-18,2.036349e-16,0,0,9.571738e-17,6.359900e-16,0,0,4.429056e-16,6.296428e-17,0,1.384528e-14,0,-1.078745e-14,0,0,0,0,0,0,0,0,0,0
7,0,0,0,2.204258e-15,0,3.528606e-14,1.936585e-15,0,9.227711e-16,2.299219e-15,2.296101e-15,9.459617e-16,1.615878e-15,1.718549e-16,0,3.214685e-16,0,1.024981e-12,0,0,0,8.893143e-17,3.473294e-15,0,0,1.647933e-15,1.089507e-14,0,0,7.514679e-15,1.051578e-15,0,1.312917e-13,0,-1.681863e-13,0,0,0,0,0,0,0,0,0,0
8,0,0,0,-3.310893e-15,0,-4.784115e-14,-2.677251e-15,0,-1.210736e-15,-3.369185e-15,-3.090096e-15,-1.403324e-15,-2.097590e-15,6.007695e-17,0,-5.656886e-16,0,-1.226033e-12,0,0,0,-1.416723e-16,-4.714540e-15,0,0,-2.504289e-15,-1.372340e-14,0,0,-9.685827e-15,-1.344527e-15,0,-1.334541e-13,0,7.890404e-14,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0.000000e+00,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0,0.000000e+00,0,0,0,0,0,0,0,0,0,0


In [62]:
# GET BUS ANGLE

res_va_degree = pd.read_csv(out_path + "/res_bus/va_degree.csv")
va_degree = res_va_degree.drop(columns=["Unnamed: 0"])

swap_slack(va_degree, slack_bus)

va_degree.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0.0,2.941895,3.184239,5.839644,7.892498,5.493763,9.013288,6.887769,8.517373,5.590963,-1.817442,-0.848353,-1.657972,-2.027026,-1.609657,-1.197858,-6.783088,-6.210950,10.210653,-4.945746,1.006267,10.338477,-2.830685,-2.819495,-1.952489,0.626103,-6.577209,7.838006,-3.709361e-07,-1.820604,-1.853158,-1.792016,-2.270649,-2.235885,0.856525,9.020372,10.338477,10.210654,7.834417,-2.819485,-6.226299,-6.577209,-6.577209,-6.577209,-1.853158
1,0.0,3.914935,4.243497,7.059837,9.098751,6.611214,10.296564,8.144127,9.749942,6.844972,-1.932661,-0.743110,-1.463974,-2.008964,-1.658310,-0.878069,-5.904249,-5.238489,11.450273,-4.491482,1.802265,11.579424,-2.578693,-1.932062,-1.930523,1.294506,-5.909768,9.005396,-1.464067e-06,-1.561452,-1.758714,-1.636016,-2.234222,-2.217283,1.780569,10.303263,11.579424,11.450275,9.001597,-1.932012,-5.252604,-5.909768,-5.909768,-5.909768,-1.758714
2,0.0,4.516446,4.910090,7.796041,9.880503,7.242007,11.115236,8.940725,10.545306,7.658409,-2.105147,-0.719268,-1.350020,-2.012654,-1.747160,-0.695485,-5.230902,-4.456969,12.244796,-4.301755,2.323995,12.374412,-2.617009,-1.239403,-2.039451,1.712391,-5.479005,9.767096,-1.138700e-06,-1.400294,-1.758301,-1.547462,-2.167352,-2.169381,2.430846,11.121713,12.374412,12.244798,9.763296,-1.239370,-4.469725,-5.479005,-5.479005,-5.479005,-1.758301
3,0.0,4.683992,5.118691,8.014756,10.163417,7.382508,11.408479,9.217806,10.846019,7.956474,-2.091461,-1.109244,-1.287012,-2.018805,-1.818798,-0.603730,-5.385255,-4.421038,12.517548,-4.809778,2.471796,12.647148,-3.060197,-1.128972,-2.127305,1.831269,-6.056743,10.022877,-1.116787e-06,-1.309620,-2.171968,-1.499276,-2.114432,-2.129229,2.615273,11.414772,12.647148,12.517550,10.019092,-1.128952,-4.433067,-6.056743,-6.056743,-6.056743,-2.171968
4,0.0,4.456765,4.890984,7.722004,9.975190,7.051827,11.199558,8.982190,10.652428,7.731385,-2.113696,-1.113784,-1.321007,-2.024006,-1.822331,-0.676921,-5.369170,-4.430457,12.325839,-4.765485,2.314756,12.455488,-3.043678,-1.197691,-2.131371,1.674270,-5.981517,9.844499,-1.536647e-06,-1.352325,-2.174475,-1.527183,-2.110650,-2.126507,2.484789,11.205957,12.455488,12.325841,9.840820,-1.197659,-4.442134,-5.981517,-5.981517,-5.981517,-2.174475
5,0.0,4.235733,4.651123,7.437938,9.755250,6.748723,10.964302,8.728178,10.423792,7.478575,-2.128744,-1.134944,-1.365660,-2.024457,-1.806552,-0.755529,-5.140094,-4.307405,12.113497,-4.466714,2.166705,12.243169,-2.913951,-1.194536,-2.110403,1.523161,-5.494013,9.651028,-3.421067e-07,-1.407767,-2.146116,-1.563178,-2.115365,-2.129826,2.369124,10.970864,12.243169,12.113498,9.647455,-1.194530,-4.318855,-5.494013,-5.494013,-5.494013,-2.146116
6,0.0,3.884481,4.261527,7.018286,9.343441,6.349137,10.552747,8.310322,10.007227,7.056027,-2.149189,-1.036520,-1.455598,-2.022288,-1.754857,-0.896263,-5.321833,-4.566582,11.718688,-4.271447,1.883825,11.848340,-2.551246,-1.484641,-2.044112,1.275349,-5.458790,9.270952,-1.418045e-06,-1.518611,-1.966909,-1.634789,-2.141781,-2.149352,2.058558,10.559436,11.848340,11.718690,9.267410,-1.484610,-4.578273,-5.458790,-5.458790,-5.458790,-1.966909
7,0.0,3.343539,3.675579,6.344639,8.734104,5.698742,9.934249,7.670282,9.388529,6.416814,-1.996267,-1.059708,-1.557082,-2.020809,-1.697342,-1.045883,-5.774554,-5.075991,11.124992,-4.414999,1.458417,11.254665,-2.546225,-1.986562,-1.970914,0.916379,-5.743661,8.697762,-1.130195e-06,-1.647057,-1.942214,-1.715448,-2.173806,-2.173147,1.575380,9.941112,11.254665,11.124993,8.694269,-1.986531,-5.088061,-5.743661,-5.743661,-5.743661,-1.942214
8,0.0,3.257056,3.592819,6.216971,8.673994,5.560928,9.866298,7.584830,9.324659,6.341185,-1.935737,-0.980251,-1.594034,-2.020259,-1.663993,-1.086450,-5.766572,-5.077946,11.063537,-4.380632,1.402693,11.193189,-2.508557,-1.957327,-1.930155,0.854210,-5.713185,8.642415,-1.290182e-06,-1.698261,-1.880726,-1.744256,-2.199247,-2.194526,1.541345,9.873193,11.193189,11.063539,8.638898

In [63]:
# GET SLACK WEIGHT (AND SLACK)

slack_active = net.ext_grid.loc[net.ext_grid["in_service"] == True]

slack_weight = [1 if i in slack_active.bus.to_list() else 0 for i in total_bus_numbers]
slack_weight[0], slack_weight[slack_bus] = slack_weight[slack_bus], slack_weight[0]

slack = slack_weight # Slack yes or no

print(slack_weight)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]


In [65]:
# GET EMPTY BUSES

active_bus = set(net.load.bus.to_list())
active_bus.update(gen_active.bus.to_list())
active_bus.update(sgen_active.bus.to_list())
active_bus.update(slack_active.bus.to_list())

empty_bus = [bus for bus in total_bus_numbers if bus not in active_bus]

for i in range(len(empty_bus)):
    if empty_bus[i] == 0:
        empty_bus[i] = slack_bus
    elif empty_bus[i] == slack_bus:
        empty_bus[i] = 0

print(empty_bus)

[35, 1, 7, 20, 27, 36]


In [66]:
# GET BUS TYPE

# 2 if PV (or slack), 3 if PQ (or empty/not in service --> P=0, Q=0)
bus_type = pd.Series([2 if (i in bus_gens.index or i in slack_active.bus.to_list()) 
                   else 3 for i in total_bus_numbers]) 


bus_type[0], bus_type[slack_bus] = 1, bus_type[0] # Main slack is 1

print(bus_type)

0     1
1     3
2     2
3     2
4     3
5     3
6     2
7     3
8     2
9     2
10    2
11    2
12    2
13    2
14    2
15    2
16    2
17    3
18    2
19    3
20    3
21    2
22    2
23    2
24    2
25    2
26    2
27    3
28    2
29    2
30    2
31    3
32    3
33    2
34    3
35    3
36    3
37    2
38    2
39    2
40    2
41    2
42    2
43    2
44    2
dtype: int64


In [67]:
# JOIN LOAD AND SGEN

load_sgen_p_mw = load_p_mw.subtract(sgen_p_mw)
load_sgen_q_mvar = load_q_mvar.subtract(sgen_q_mvar)

load_sgen_p_mw.head(5)

bus,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
0,0,0,182.537141,84.322491,152.095808,-325.067357,-18.789241,0,33.560606,154.378892,491.685974,-31.852280,57.756558,-11.448294,155.276240,153.391824,313.684397,917.941047,0,249.645013,0,-564.805929,1116.089935,124.323587,220.364614,-372.431501,857.816644,0,194.095314,1240.037646,255.650291,141.042212,1190.143366,287.117430,376.240487,0,0,0,0,0,0,0,0,0,0
1,0,0,182.236715,64.666964,151.845483,-322.698322,-63.569805,0,41.642623,135.684118,453.690784,-37.936396,58.704608,-11.807279,155.020681,135.572848,313.168123,846.715032,0,249.234138,0,-587.079077,1017.274144,124.118971,207.646237,-389.898263,813.084907,0,193.775865,1169.429508,255.811001,140.810079,1112.054024,254.248531,334.262028,0,0,0,0,0,0,0,0,0,0
2,0,0,182.081322,48.286803,151.716005,-301.145974,-88.698956,0,44.982938,116.639738,412.615002,-50.322963,63.586702,-11.157840,154.888495,119.797198,312.901086,767.567011,0,249.021616,0,-594.880740,908.366273,124.013135,195.154303,-384.266837,763.114219,0,193.610633,1089.698274,255.293492,140.690011,1016.204466,221.742408,289.887785,0,0,0,0,0,0,0,0,0,0
3,0,0,181.698019,38.334455,151.396625,-277.320328,-101.391451,0,38.208470,102.187084,382.496477,-62.587140,66.273809,-10.828764,154.562436,108.561068,312.242392,707.782406,0,248.497397,0,-594.617733,833.628198,123.752073,184.830506,-366.739036,724.220248,0,193.203059,1032.707549,257.863582,140.393842,948.523995,195.284015,256.363307,0,0,0,0,0,0,0,0,0,0
4,0,0,182.174558,38.167862,151.793692,-251.230436,-94.159785,0,35.102811,99.116672,377.965018,-61.574570,68.239287,-10.096669,154.967806,109.095681,313.061308,698.732888,0,249.149129,0,-595.436474,827.401974,124.076636,181.967714,-332.831460,716.284907,0,193.709772,1028.190987,263.033624,140.762052,943.031432,187.018997,249.700721,0,0,0,0,0,0,0,0,0,0


In [68]:
# GET MAX AND MIN GEN MVAR

min_gen_q_mvar = gen_active.groupby("bus")["min_q_mvar"].sum()
min_gen_q_mvar = min_gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), fill_value=None) # Fill missing buses with none

max_gen_q_mvar = gen_active.groupby("bus")["max_q_mvar"].sum()
max_gen_q_mvar = max_gen_q_mvar.reindex(range(total_bus_numbers[0], total_bus_numbers[-1] + 1), fill_value=None) # Fill missing buses with none

min_gen_q_mvar

bus
0        NaN
1        NaN
2    -1000.0
3    -2000.0
4        NaN
5        NaN
6    -4000.0
7        NaN
8    -2000.0
9    -1000.0
10   -3000.0
11   -4000.0
12   -4000.0
13   -1000.0
14   -1000.0
15   -4000.0
16   -1000.0
17       NaN
18   -2000.0
19       NaN
20       NaN
21   -4000.0
22   -5000.0
23   -2000.0
24   -3000.0
25   -2000.0
26   -1000.0
27       NaN
28   -7000.0
29   -3000.0
30   -2000.0
31       NaN
32       NaN
33   -1000.0
34       NaN
35       NaN
36       NaN
37       NaN
38       NaN
39       NaN
40       NaN
41       NaN
42       NaN
43       NaN
44       NaN
Name: min_q_mvar, dtype: float64

In [69]:
# GET USER SOLAR AND WIND BUSSES

extra_bus = pd.Series([2, None, None, None, None, 0, 0, 1000.0, -1000.0, 0, 0])
extra_buses = pd.DataFrame([extra_bus, extra_bus])
extra_buses.columns = ["Type", "PU Volt", "Angle (Deg)", "Gen MW", "Gen Mvar", "Load MW", "Load Mvar", 
                       "Gen Mvar(max)", "Gen Mvar(min)", "Slack yes,no", "Slack Weight"]
extra_buses

,Type,PU Volt,Angle (Deg),Gen MW,Gen Mvar,Load MW,Load Mvar,Gen Mvar(max),Gen Mvar(min),"Slack yes,no",Slack Weight
0,2.0,NaN,NaN,NaN,NaN,0.0,0.0,1000.0,-1000.0,0.0,0.0
1,2.0,NaN,NaN,NaN,NaN,0.0,0.0,1000.0,-1000.0,0.0,0.0


In [70]:
# TRANSFORM TO P.U.

def to_pu(power, sbase=100.0):
    return np.divide(power, sbase)

In [71]:
# GETTING ALL DATA FOR ONE TIMESTEP

nones = [None for _ in total_bus_numbers]

def get_timestep(t):
    timestep = pd.DataFrame(data={"Type": bus_type, "PU Volt": vm_pu.iloc[t].values, "Angle (Deg)": va_degree.iloc[t].values, 
    "Gen MW": to_pu(gen_p_mw.iloc[t].values), "Gen Mvar": nones, "Load MW": to_pu(load_sgen_p_mw.iloc[t].values), 
    "Load Mvar": to_pu(load_sgen_q_mvar.iloc[t].values), "Gen Mvar(max)": to_pu(max_gen_q_mvar), 
    "Gen Mvar(min)": to_pu(min_gen_q_mvar), "Slack yes,no": slack, "Slack Weight": slack_weight})
    
    timestep = pd.concat([timestep, extra_buses], axis=0, ignore_index=True) # Add solar and wind bus
    
    return timestep

get_timestep(0).head(50)


,Type,PU Volt,Angle (Deg),Gen MW,Gen Mvar,Load MW,Load Mvar,Gen Mvar(max),Gen Mvar(min),"Slack yes,no",Slack Weight
0,1.0,1.050000,0.000000e+00,0.000000e+00,None,0.000000,0.000000e+00,NaN,NaN,1.0,1.0
1,3.0,1.048237,2.941895e+00,0.000000e+00,None,0.000000,0.000000e+00,NaN,NaN,0.0,0.0
2,2.0,1.049461,3.184239e+00,7.115815e-10,None,1.825371,5.999706e-01,10.0,-10.0,0.0,0.0
3,2.0,1.050000,5.839644e+00,4.461269e-11,None,0.843225,8.911029e-01,20.0,-20.0,0.0,0.0
4,3.0,1.047128,7.892498e+00,0.000000e+00,None,1.520958,4.999147e-01,NaN,NaN,0.0,0.0
5,3.0,1.049809,5.493763e+00,0.000000e+00,None,-3.250674,2.096368e-01,NaN,NaN,0.0,0.0
6,2.0,1.049990,9.013288e+00,8.774756e-10,None,-0.187892,1.082659e+00,40.0,-40.0,0.0,0.0
7,3.0,1.049905,6.887769e+00,0.000000e+00,None,0.000000,0.000000e+00,NaN,NaN,0.0,0.0
8,2.0,1.049701,8.517373e+00,5.000000e-01,None,0.335606,3.085760e-01,20.0,-20.0,0.0,0.0
9,2.0,1.050000,5.590963e+00,4.800000e-01,None,1.543789,5.669805e-01,10.0,-10.0,0.0,0.0


In [73]:
# CREATING THE CSV WITH ALL TIMESTEPS

def create_csv(t_init, t_duration, index=True):
    result = pd.concat([get_timestep(t) for t in range(t_init, t_init + t_duration)], axis=0)
    
    result.to_csv(out_path + "/busInputData.csv", index=index)

In [80]:
# GET NON-CONVERGED TIMESTEPS INDEXES

non_conv_mask = (vm_pu == 0).all(axis=1)
non_conv_idx = vm_pu.index[non_conv_mask].to_list() # Non converged timesteps index

conv_periods = [non_conv_idx[i+1] - non_conv_idx[i] for i in range(len(non_conv_idx) - 1)] # Periods of convergence
from_to_conv = [[non_conv_idx[i], non_conv_idx[i+1]] for i in range(len(non_conv_idx) - 1)]
convergences = dict(zip(conv_periods, from_to_conv))

conv_periods_sorted = conv_periods
conv_periods_sorted.sort(reverse=True)
conv = {i: convergences[i] for i in conv_periods_sorted} # {Length period of convergence: [start, end]}

print(conv)

{186: [7982, 8168], 149: [4829, 4978], 141: [8364, 8505], 131: [6770, 6901], 121: [5959, 6080], 113: [7391, 7504], 112: [7746, 7858], 110: [5775, 5885], 106: [728, 834], 105: [6080, 6185], 104: [6911, 7015], 103: [136, 239], 100: [6406, 6506], 98: [3519, 3617], 96: [1329, 1425], 93: [4173, 4266], 91: [8271, 8362], 89: [1885, 1974], 84: [1069, 1153], 82: [7266, 7348], 80: [3711, 3791], 79: [3440, 3519], 77: [6563, 6640], 76: [8511, 8587], 75: [7015, 7090], 74: [7136, 7210], 72: [5141, 5213], 70: [426, 496], 68: [7911, 7979], 63: [5078, 5141], 62: [7576, 7638], 61: [2205, 2266], 60: [8587, 8647], 59: [3835, 3894], 56: [7210, 7266], 55: [1561, 1616], 54: [9, 63], 53: [7638, 7691], 52: [6326, 6378], 51: [8220, 8271], 50: [4743, 4793], 48: [7691, 7739], 47: [2933, 2980], 46: [919, 965], 45: [3234, 3279], 44: [7504, 7548], 43: [7348, 7391], 42: [2987, 3029], 41: [5653, 5694], 40: [4357, 4397], 38: [4077, 4115], 37: [2474, 2511], 36: [4041, 4077], 34: [7877, 7911], 33: [6506, 6539], 32: [6185

In [81]:
create_csv(6912, 48, index=False) # 6912/24 = 288th day 1st hour

In [86]:
load_p_mw.loc[6912:6912+48]

bus,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
6912,0,0,184.774800,194.042552,153.960297,64.562503,247.228259,0,95.033123,105.918116,374.073824,177.913665,285.542397,117.884773,157.179716,219.079873,317.529743,705.913250,0,252.705323,0,0,914.847691,125.847626,182.968558,148.455061,702.369036,0,196.474660,1189.367712,326.231379,142.771198,1127.555085,185.502541,256.281400,0,0,0,0,0,0,0,0,0,0
6913,0,0,185.468888,189.550898,154.538634,64.805025,242.470070,0,95.390106,101.888276,365.392054,175.014164,286.615009,118.327596,157.770147,216.638302,318.722512,689.615014,0,253.654586,0,0,892.537180,126.320360,181.304344,145.173863,693.500929,0,197.212698,1167.965964,327.456835,143.307504,1104.954841,180.033735,246.782497,0,0,0,0,0,0,0,0,0,0
6914,0,0,185.613922,186.505806,154.659480,64.855702,239.000902,0,95.464699,99.330671,359.190409,172.767503,286.839137,118.420126,157.893520,215.500335,318.971748,678.463338,0,253.852939,0,0,877.355627,126.419141,180.504386,142.778619,686.972861,0,197.366915,1149.588508,327.712901,143.419568,1085.965473,177.705312,240.674788,0,0,0,0,0,0,0,0,0,0
6915,0,0,186.173336,187.190540,155.125602,65.051168,239.963344,0,95.752417,99.690152,360.687336,173.485601,287.703630,118.777027,158.369389,215.816740,319.933084,680.990911,0,254.618017,0,0,880.649365,126.800150,180.808473,143.389715,689.321750,0,197.961751,1155.644511,328.700582,143.851815,1091.824256,177.611804,241.580223,0,0,0,0,0,0,0,0,0,0
6916,0,0,188.410994,193.538088,156.990091,65.833032,247.472194,0,96.903286,104.293423,373.138077,178.503875,291.161603,120.204635,160.272866,220.391255,323.778431,703.999845,0,257.678327,0,0,911.349012,128.324189,184.408302,148.231679,706.597696,0,200.341097,1192.287426,332.651307,145.580802,1128.364486,183.487208,252.567729,0,0,0,0,0,0,0,0,0,0
6917,0,0,198.749805,225.856049,165.604720,69.445535,286.000262,0,102.220729,127.871756,437.475922,204.387327,307.138721,126.800709,169.067633,241.181138,341.545354,821.509516,0,271.818093,0,0,1068.398714,135.365814,200.791558,173.324096,793.125531,0,211.334556,1387.643704,350.905118,153.569360,1323.722298,209.980134,309.018037,0,0,0,0,0,0,0,0,0,0
6918,0,0,206.581609,307.711045,172.130431,72.182060,384.808111,0,106.248772,191.593022,609.730135,271.220147,319.241627,131.797334,175.729801,265.505916,355.004066,1122.911767,0,282.529178,0,0,1475.772279,140.699950,219.379724,240.921722,986.489910,0,219.662267,1957.864201,364.732655,159.620814,1903.533726,246.246458,462.913820,0,0,0,0,0,0,0,0,0,0
6919,0,0,209.140413,383.584516,174.262508,73.076136,478.086968,0,107.564812,251.145219,774.277772,335.073741,323.195884,133.429830,177.906461,275.496466,359.401291,1404.135603,0,286.028699,0,0,1857.012484,142.442717,226.924586,305.875696,1159.422568,0,222.383093,2532.792257,369.250381,161.597942,2489.648262,261.954259,607.640693,0,0,0,0,0,0,0,0,0,0
6920,0,0,209.865580,406.108382,174.866741,73.329518,505.505620,0,107.937779,268.938542,822.645115,353.717425,324.316523,133.892480,178.523329,279.465036,360.647468,1487.451738,0,287.020466,0,0,1969.971286,142.936618,229.881990,324.914489,1210.650507,0,223.154178,2698.216890,370.530709,162.158262,2658.443370,268.553743,650.768219,0,0,0,0,0,0,0,0,0,0
6921,0,0,209.679108,416.439758,174.711367,73.264363,518.484004,0,107.841873,277.069993,845.759593,362.725626,324.028359,133.773513,178.364706,279.071406,360.327022,1526.008914,0,286.765440,0,0,2022.369067,142.809615,229.573057,334.097589,1233.499622,0,222.955899,2783.439627,370.201482,162.014179,2745.440883,268.040667,670.667291,0,0,0,0,0,0,0,0,0,0


In [103]:
net.ext_grid

,name,bus,vm_pu,va_degree,slack_weight,in_service,min_p_mw,max_p_mw,min_q_mvar,max_q_mvar,controllable,type,p_mw
0,380kV Germany (Diele) - Groningen,38,1.049999,0.000000,1.0,True,-1972.0,1972.0,-5000.0,5000.0,True,External Grid,0.0
1,380kV Germany (Gronau) - Enschede,39,1.050000,4.244105,1.0,True,-1972.0,1972.0,-5000.0,5000.0,True,External Grid,0.0
2,380kV Germany (Wesel) - Doetichem,40,1.049989,4.129643,1.0,True,-1972.0,1972.0,-5000.0,5000.0,True,External Grid,0.0
3,380kV Germany (Rommerskirchen) - Roermond,41,1.050000,2.372583,1.0,True,-986.0,986.0,-5000.0,5000.0,True,External Grid,0.0
4,380kV Germany (Siersdorf) - Roermond,42,1.049943,-5.248131,1.0,True,-986.0,986.0,-5000.0,5000.0,True,External Grid,0.0
5,380kV Belgium (Van Eyck) - Maasbracht,43,1.047887,-7.856319,1.0,True,-1128.0,1128.0,-5000.0,5000.0,True,External Grid,0.0
6,380kV Belgium (Zandvliet) - Rilland,44,1.047047,-7.099175,1.0,True,-1128.0,1128.0,-5000.0,5000.0,True,External Grid,0.0
7,380kV Britain (BritNed) - Maasvlakte,35,1.047047,-7.099176,1.0,True,-1000.0,1000.0,-5000.0,5000.0,True,External Grid,0.0
8,380kV Denmark (COBRA) - Eemshaven,36,1.047047,-7.099176,1.0,False,-700.0,700.0,-5000.0,5000.0,True,External Grid,0.0
9,380kV Norway (NorNed) - Eemshaven,37,1.049581,-2.560716,1.0,True,-700.0,700.0,-5000.0,5000.0,True,External Grid,0.0


In [104]:
# TESTING OPF

res_load_p_mw = pd.read_csv(out_path + "/res_load/p_mw.csv") # Read from simulation result
res_load_q_mvar = res_load_p_mw.mul(p_to_q)

res_gen_vm_pu = pd.read_csv(out_path + "/res_gen/vm_pu.csv")

res_ext_grid_vpu = res_vm_pu.iloc[:, 36:] # Wrong
res_ext_grid_vm_pu = pd.concat([res_ext_grid_vpu.iloc[:, 3:], res_ext_grid_vpu.iloc[:, :3]], axis=1)

res_ext_grid_vadeg = res_va_degree.iloc[:, 36:]
res_ext_grid_va_degree = pd.concat([res_ext_grid_vadeg.iloc[:, 3:], res_ext_grid_vadeg.iloc[:, :3]], axis=1)
# res_ext_grid_vm_pu.head()


def run_simulation(t_init=0, t_final=8760, function=pp.runpp):

    # init_load = net.load[["p_mw", "q_mvar"]]

    net.ext_grid["vm_pu"] = np.ones(len(net.ext_grid))
    net.ext_grid["va_degree"] = np.zeros(len(net.ext_grid))

    res_lp = []
    res_lq = []

    res_gp = []
    res_gq = []
    res_gvm = []
    res_gva = []

    res_sgp = []
    res_sgq = []

    res_bp = []
    res_bq = []
    res_bvm = []
    res_bva = []

    res_l = []

    for t in range(t_init, t_final):
        net.load["p_mw"] = res_load_p_mw.iloc[t, 1:].values
        net.load["q_mvar"] = res_load_q_mvar.iloc[t, 1:].values

        net.gen["p_mw"] = res_gen_p_mw.iloc[t, 1:].values
        net.gen["vm_pu"] = res_gen_vm_pu.iloc[t, 1:].values

        net.sgen["p_mw"] = res_sgen_p_mw.iloc[t, 1:].values
        net.sgen["q_mvar"] = res_sgen_q_mvar.iloc[t, 1:].values

        # print(res_ext_grid_vm_pu.iloc[t].values)

        net.ext_grid["vm_pu"] = res_ext_grid_vm_pu.iloc[t].values
        net.ext_grid["va_degree"] = res_ext_grid_va_degree.iloc[t].values

        function(net)

        res_lp.append(net.res_load["p_mw"].copy().to_frame().T) # load_p_mw
        res_lq.append(net.res_load["q_mvar"].copy().to_frame().T) # load_q_mvar

        res_gp.append(net.res_gen["p_mw"].copy().to_frame().T) # gen_p_mw
        res_gq.append(net.res_gen["q_mvar"].copy().to_frame().T) # gen_q_mvar
        res_gvm.append(net.res_gen["vm_pu"].copy().to_frame().T) # gen_vm_pu
        res_gva.append(net.res_gen["va_degree"].copy().to_frame().T) # gen_q_mvar

        res_sgp.append(net.res_sgen["p_mw"].copy().to_frame().T) # gen_p_mw
        res_sgq.append(net.res_sgen["q_mvar"].copy().to_frame().T) # gen_p_mw

        res_bp.append(net.res_bus["p_mw"].copy().to_frame().T) # gen_q_mvar
        res_bq.append(net.res_bus["q_mvar"].copy().to_frame().T) # gen_q_mvar
        res_bvm.append(net.res_bus["vm_pu"].copy().to_frame().T) # gen_q_mvar
        res_bva.append(net.res_bus["va_degree"].copy().to_frame().T) # gen_q_mvar

        res_l.append(net.res_line["loading_percent"].copy().to_frame().T) # gen_q_mvar

    
    res_lp = pd.concat(res_lp, axis=0)
    res_lq = pd.concat(res_lq, axis=0)

    res_gp = pd.concat(res_gp, axis=0)
    res_gq = pd.concat(res_gq, axis=0)
    res_gvm = pd.concat(res_gvm, axis=0)
    res_gva = pd.concat(res_gva, axis=0)

    res_sgp = pd.concat(res_sgp, axis=0)
    res_sgq = pd.concat(res_sgq, axis=0)

    res_bp = pd.concat(res_bp, axis=0)
    res_bq = pd.concat(res_bq, axis=0)
    res_bvm = pd.concat(res_bvm, axis=0)
    res_bva = pd.concat(res_bva, axis=0)

    res_l = pd.concat(res_l, axis=0)

    return {"load": [res_lp, res_lq], "gen": [res_gp, res_gq, res_gvm, res_gva], "sgen": [res_sgp, res_sgq], 
            "bus": [res_bp, res_bq, res_bvm, res_bva], "line": [res_l]}


results = run_simulation(t_init=6912, t_final=6912+48, function=pp.runpp)


In [105]:
results["line"][0].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91
loading_percent,18.233950,18.233950,1.580832,1.580832,11.659965,11.659965,2.898469,2.898469,1.987191,1.987191,5.520081,5.520081,14.571593,14.527192,14.527192,8.340281,8.340281,4.165912,4.165912,5.612250,5.612250,1.152192,11.026057,11.026057,1.304492,1.304492,9.709734,9.709734,1.528564,1.528564,10.011686,10.011686,1.045385,1.045385,3.360951,3.360951,23.252188,23.252188,2.437585,2.437585,17.162399,17.162399,6.954519,6.954519,30.142651,30.142651,0.237214,0.237214,17.902232,17.902232,17.902232,3.487137,3.487137,4.257628,2.546404,2.546404,13.602280,13.602280,1.161652,3.250892,3.250892,5.934682,5.934682,17.311775,17.311775,0.145714,0.145714,0.392820,0.392820,10.339584,10.339584,1.253589,0.095489,0.095489,0.116904,0.116904,18.772271,18.772271,3.017186,3.017186,6.466725,6.466725,24.683051,24.683051,15.663267,15.663267,0.001809,0.001809,0.000905,0.000905,0.014955,0.014955
loading_percent,17.559955,17.559955,1.678299,1.678299,11.614363,11.614363,3.009403,3.009403,1.736706,1.736706,5.439425,5.439425,14.317521,14.275402,14.275402,8.038684,8.038684,4.257582,4.257582,5.512645,5.512645,1.136149,10.752762,10.752762,1.175143,1.175143,9.423780,9.423780,1.390250,1.390250,9.488018,9.488018,1.043797,1.043797,3.373837,3.373837,22.826321,22.826321,2.365634,2.365634,16.765755,16.765755,6.866590,6.866590,29.298525,29.298525,0.237187,0.237187,17.746960,17.746960,17.746960,3.219362,3.219362,3.881888,2.322789,2.322789,13.055100,13.055100,1.164285,3.271462,3.271462,5.931987,5.931987,17.081433,17.081433,0.145716,0.145716,0.437273,0.437273,10.497075,10.497075,0.853399,0.095549,0.095549,0.116903,0.116903,18.599088,18.599088,2.669798,2.669798,6.292027,6.292027,24.252527,24.252527,15.294711,15.294711,0.001809,0.001809,0.002025,0.002025,0.129480,0.129480
loading_percent,17.256068,17.256068,1.791888,1.791888,11.961747,11.961747,3.203115,3.203115,1.368100,1.368100,5.357513,5.357513,14.470198,14.429717,14.429717,7.904709,7.904709,4.303123,4.303123,5.487449,5.487449,1.119153,10.505172,10.505172,1.262884,1.262884,9.360288,9.360288,1.359558,1.359558,8.940545,8.940545,1.033519,1.033519,3.372632,3.372632,22.532125,22.532125,2.316245,2.316245,16.616712,16.616712,6.643155,6.643155,28.998483,28.998483,0.237153,0.237153,17.617855,17.617855,17.617855,3.011751,3.011751,3.503595,1.991492,1.991492,12.640271,12.640271,1.185332,3.273548,3.273548,5.924203,5.924203,16.904870,16.904870,0.145705,0.145705,0.487919,0.487919,10.676087,10.676087,0.275742,0.095488,0.095488,0.116897,0.116897,18.415123,18.415123,2.361599,2.361599,6.141454,6.141454,23.952292,23.952292,15.160935,15.160935,0.001809,0.001809,0.000906,0.000906,0.018238,0.018238
loading_percent,17.453758,17.453758,1.987572,1.987572,12.123150,12.123150,3.175176,3.175176,1.121131,1.121131,5.405465,5.405465,14.460286,14.418941,14.418941,7.728679,7.728679,4.207112,4.207112,5.259115,5.259115,1.113543,10.519124,10.519124,1.424544,1.424544,9.357539,9.357539,1.350440,1.350440,8.855882,8.855882,1.038423,1.038423,3.342693,3.342693,22.575699,22.575699,2.326611,2.326611,16.547680,16.547680,6.938577,6.938577,29.041899,29.041899,0.237156,0.237156,17.721952,17.721952,17.721952,3.038302,3.038302,3.449926,1.827380,1.827380,12.729735,12.729735,1.191475,3.547947,3.547947,5.989194,5.989194,17.024239,17.024239,0.145707,0.145707,0.499590,0.499590,10.721194,10.721194,0.248303,0.095489,0.095489,0.116898,0.116898,18.359242,18.359242,2.300812,2.300812,6.159402,6.159402,24.008148,24.008148,15.049607,15.049607,0.001809,0.001809,0.000906,0.000906,0.018954,0.018954
loading_percent,17.868504,17.868504,1.888156,1.888156,11.896727,11.896727,2.758555,2.758555,1.028140,1.028140,5.525884,5.525884,14.191139,14.147421,14.147421,7.579588,7.579588,4.052958,4.052958,4.857863,4.857863,1.140058,11.028759,11.0